In [2]:
from dotenv import load_dotenv 
load_dotenv()

True

In [6]:
from langchain.document_loaders import DirectoryLoader 
from langchain.document_loaders import PyPDFLoader 
data_folder = "../data"  

loader = DirectoryLoader(data_folder, glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()


In [7]:
len(documents)

436

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=150)

In [13]:
chunks=splitter.split_documents(documents)

In [14]:
len(chunks)

6215

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\saipr\AppData\Local\Temp\ipykernel_9948\3920331017.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\saipr\BioRag\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
from langchain.vectorstores import FAISS 
vectorstore=FAISS.from_documents(chunks,embeddings)


In [41]:
retriver=vectorstore.as_retriever()

In [45]:
retriver = vectorstore.as_retriever(search_kwargs={"k": 5})


In [25]:
query="what is Endoplasmic reticulum (ER)"
docc=retriver.get_relevant_documents(query,k=10)

In [26]:
docc

[Document(id='64bebd7c-963a-4417-afa8-4a9f847795d0', metadata={'producer': '3-Heights(TM) PDF Optimization Shell 4.8.25.2 (http://www.pdf-tools.com)', 'creator': 'PageMaker 7.0', 'creationdate': '2017-01-18T18:28:08+05:30', 'author': 'Admin', 'title': 'IX Bio_Title Free.pmd', 'moddate': '2019-05-28T07:37:38+00:00', 'source': '..\\data\\ix biology em.pdf', 'total_pages': 194, 'page': 17, 'page_label': '18'}, page_content='Cell  its structure and functions6\nsubstances from one part of the cell to\nanother. This network of membranes is\nknown as the endoplasmic reticulum.\nThe endoplasmic reticulum (ER) is a\nlarge network of membrane-bound tubes\nand sheets. The ER membrane is similar in\nstructure to the plasma membrane.\nEndoplasmic reticulum may have some\ngranule like structure on its surface which\nare called ribosomes, Such ER is called\nrough endoplasmic reticulum (RER).\nAreas/sections of ER that do not have'),
 Document(id='1b74ad9c-9271-4b6c-9159-7a0ae3fac413', metadata={'prod

In [27]:
from langchain_groq import ChatGroq 
import os 
GROQ_API_KEY=os.getenv('GROQ_API_KEY')

llm=ChatGroq(api_key=GROQ_API_KEY,model='llama-3.3-70b-versatile')

In [53]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

retriever_dense = vectorstore.as_retriever(search_kwargs={"k": 10})  
bm25_retriever = BM25Retriever.from_documents(chunks)

hybrid_retriever = EnsembleRetriever(retrievers=[retriever_dense, bm25_retriever], weights=[0.5, 0.5])


In [63]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and knowledgeable biology tutor. Answer clearly and accurately.if the query is out of sylabus just give Out of syllabus"),
    ("human", "Context:\n{context}\n\nQuestion: {question}")
])


In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=hybrid_retriever,  
    return_source_documents=True,
    chain_type="stuff",  
    chain_type_kwargs={"prompt": prompt}  )

query = "Who is virat kohli"
result = qa_chain({"query": query})
print(result['result'])

Out of syllabus


In [50]:
from sentence_transformers import CrossEncoder
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.schema import Document
from typing import List

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rerank_documents(query: str, retrieved_docs: List[Document]) -> List[Document]:
    """Re-rank retrieved documents using a Cross-Encoder."""
    docs_texts = [doc.page_content for doc in retrieved_docs]
    scores = reranker.predict([(query, doc) for doc in docs_texts])
    
    sorted_docs = [doc for _, doc in sorted(zip(scores, retrieved_docs), key=lambda x: x[0], reverse=True)]
    return sorted_docs

query = "Explain the parts of human brain and their functions"
retrieved_docs = retriver.get_relevant_documents(query,k=5)  
reranked_docs = rerank_documents(query, retrieved_docs)  

qa_chain = load_qa_chain(llm, chain_type="stuff")

final_answer = qa_chain.run(input_documents=reranked_docs, question=query)


print("\n💡 AI Answer:", final_answer)



💡 AI Answer: The human brain can be divided into several parts, each with distinct functions. Here's an overview of the main parts of the brain and their functions:

1. **Forebrain**: The forebrain is the most advanced part of the brain and is responsible for higher-level cognitive functions such as thought, emotion, and movement. It consists of two main parts:
	* **Cerebrum**: The cerebrum is the largest part of the brain and is divided into two hemispheres: the left hemisphere and the right hemisphere. The cerebrum is responsible for processing sensory information, controlling movement, and managing higher-level cognitive functions such as thought, emotion, and memory.
	* **Diencephalon**: The diencephalon is a small region that connects the cerebrum to the brainstem. It plays a role in relaying sensory information to the cerebrum and regulating body functions such as sleep, hunger, and thirst.
2. **Midbrain**: The midbrain is a small region that connects the forebrain to the hindbr